In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json

# A function that translates the json_string into a unicode string
def jsonreader(exstring):
    ind_s = exstring.index("('")+2
    ind_e = exstring.index("')")
    json_data = exstring[ind_s:ind_e]
    return json.loads(json_data.encode('utf8').decode('unicode_escape'))

#I found that the first game of the season was Brentford 2 - 0 Arsenal, looked for that url. Then 
# tested higher number urls until i got to a recent game i recognized.
base_url = 'https://understat.com/match/'
matchnum = list(range(16376,16607)) 
#matches 16376->16607

urls = []

#After loading all the urls of premier league games, I wanted to find strictly the Arsenal game urls. 
# Filtering down early saves time later on. This is fine since I only want Arsenal data.
for each in matchnum:
    url = base_url+str(each)+'/'
    r = requests.get(url)
    soup = bs(r.content, 'html.parser')
    header = str(soup.find('title'))
    if header.find('Arsenal') != -1:
        urls.append(url)
        

print(urls)

#I'm defining this early. This will become the dataset that eventually becomes the dataframe for use in pandas.
seasonrosterdata = []

['https://understat.com/match/16376/', 'https://understat.com/match/16394/', 'https://understat.com/match/16396/', 'https://understat.com/match/16407/', 'https://understat.com/match/16419/', 'https://understat.com/match/16434/', 'https://understat.com/match/16436/', 'https://understat.com/match/16446/', 'https://understat.com/match/16456/', 'https://understat.com/match/16468/', 'https://understat.com/match/16476/', 'https://understat.com/match/16489/', 'https://understat.com/match/16496/', 'https://understat.com/match/16512/', 'https://understat.com/match/16517/', 'https://understat.com/match/16526/', 'https://understat.com/match/16536/', 'https://understat.com/match/16548/', 'https://understat.com/match/16562/', 'https://understat.com/match/16576/', 'https://understat.com/match/16596/']


In [2]:
# I need to find my columns. The best way to do that is work with a specific json_string and find the columns info in there.
r1 = requests.get('https://understat.com/match/16376')

soup1 = bs(r1.content, 'html.parser')

scripts1 = soup1.find_all('script')

rosters_json_string1 = ''
for each in scripts1:
    if 'rostersData' in each.text:
        rosters_json_string = each.text.strip()

json_dict1 = jsonreader(rosters_json_string)

# column and values names for an individual player
firstplayer = json_dict1['a']['475189']
columns = list(firstplayer.keys())
print(columns)
#I need the team_id for Arsenal, I can verify via a player who is definitely
print(firstplayer['player'])
print(firstplayer['team_id'])


['id', 'goals', 'own_goals', 'shots', 'xG', 'time', 'player_id', 'team_id', 'position', 'player', 'h_a', 'yellow_card', 'red_card', 'roster_in', 'roster_out', 'key_passes', 'assists', 'xA', 'xGChain', 'xGBuildup', 'positionOrder']
Ben White
83


In [3]:
#looping through all the Arsenal URLs
for url in urls:
    r = requests.get(url)
    soup = bs(r.content, 'html.parser')
    scripts = soup.find_all('script')
#Finding the player stats and changing it into a dictionary
    rosters_json_string = ''
    for each in scripts:
        if 'rostersData' in each.text:
            rosters_json_string = each.text.strip()
    json_dict = jsonreader(rosters_json_string)
    
    #making a list of lists. Every game has a home and away team, I need each of the players' values as a list
    for row in json_dict['h']:
        seasonrosterdata.append(list(json_dict['h'][row].values()))

    for row in json_dict['a']:
        seasonrosterdata.append(list(json_dict['a'][row].values()))



In [4]:
df = pd.DataFrame(seasonrosterdata,columns=columns)

In [6]:
df.head()

,id,goals,own_goals,shots,xG,time,player_id,team_id,position,player,...,yellow_card,red_card,roster_in,roster_out,key_passes,assists,xA,xGChain,xGBuildup,positionOrder
559,510322,0,0,2,0.05627342313528061,90,8089,83,DL,Kieran Tierney,...,0,0,0,0,2,0,0.06077491492033005,0.6583656668663025,0.565555989742279,4
560,510323,0,0,1,0.02423863485455513,90,9689,83,DMC,Albert Sambi Lokonga,...,0,0,0,0,1,0,0.07634542882442474,0.17820847034454346,0.10186304897069931,7
561,510324,0,0,4,0.1741524338722229,90,7322,83,MR,Bukayo Saka,...,0,0,0,0,1,0,0.04554996266961098,0.39107850193977356,0.2609484791755676,8
562,510325,0,0,3,0.24273301661014557,90,2517,83,MC,Martin Odegaard,...,0,0,0,0,4,0,0.18116991221904755,0.3607531487941742,0.12036784738302231,9
563,510326,0,0,2,0.16933895647525787,79,7230,83,MC,Emile Smith-Rowe,...,0,0,510329,0,2,0,0.4814806282520294,0.7446892261505127,0.1570543497800827,9
564,510327,0,0,2,0.0910346731543541,90,7752,83,ML,Gabriel Martinelli,...,0,0,0,0,2,0,0.08149347454309464,0.24474678933620453,0.10968969017267227,10
565,510328,0,0,3,0.5095507502555847,90,3277,83,FW,Alexandre Lacazette,...,0,0,0,0,3,0,0.2086305469274521,0.7181812524795532,0.21184876561164856,15
566,510329,0,0,0,0,11,6482,83,Sub,Eddie Nketiah,...,0,0,0,510326,0,0,0,0,0,17
567,510330,0,0,0,0,90,5552,92,GK,Nick Pope,...,0,0,0,0,0,0,0,0,0,1
568,510331,0,0,0,0,90,5568,92,DR,Connor Roberts,...,0,0,0,0,0,0,0,0,0,2


In [ ]:
##Write the DataFrame to a CSV file.
# import os
# os.makedirs('/Users/fsa168/Desktop/PythonPractice/SoccerScraper', exist_ok=True)

# df.to_csv('/Users/fsa168/Desktop/PythonPractice/SoccerScraper/full_season_data.csv')